In [1]:
import gc
gc.enable()

In [2]:
!pip install keras-adabound
!pip install segmentation-models

  Stored in directory: /tmp/.cache/pip/wheels/26/fb/78/f6aa020cb8f098fecdf1e9043a9bb259c8414692d4225c6183
Successfully built keras-adabound
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 51kB 3.5MB/s 
    100% |████████████████████████████████| 81kB 10.2MB/s 
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from keras import backend as K
from keras.models import load_model
from segmentation_models.losses import dice_loss
import os
import numpy as np
from skimage.io import imread
import pandas as pd
from keras_adabound import AdaBound

def f2_micro(y_true, y_pred):
    agreement = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    total_true_positive = K.sum(K.round(K.clip(y_true, 0, 1)))
    total_pred_positive = K.sum(K.round(K.clip(y_pred, 0, 1)))
    recall = agreement / (total_true_positive + K.epsilon())
    precision = agreement / (total_pred_positive + K.epsilon())
    return (1+2**2)*((precision*recall)/(2**2*precision+recall+K.epsilon()))

def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

extra = {
    'dice_loss': dice_loss,
    'f2_micro': f2_micro,
    'AdaBound': AdaBound
}
ship_dir = '../input/airbus-ship-detection/'
test_image_dir = os.path.join(ship_dir, 'test_v2')
test_paths = os.listdir(test_image_dir)


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [4]:
from tqdm import tqdm_notebook
from skimage.morphology import binary_opening, disk
from skimage.morphology import label
# import os

fpn = load_model("../input/airbusv6/fpn3.5.hdf5", custom_objects=extra)
unet = load_model("../input/airbusv6/unet3.5.hdf5", custom_objects=extra)
models = [fpn,unet]

out_pred_rows = [[],[]]
# i = 0
for c_img_name in tqdm_notebook(test_paths):
    c_path = os.path.join(test_image_dir, c_img_name)
    c_img = imread(c_path)
    c_img = np.expand_dims(c_img, 0)/255.0
    for m in range(len(models)):
        cur_seg = models[m].predict(c_img)[0]
        cur_seg = binary_opening(cur_seg>0.5, np.expand_dims(disk(2), -1))
        cur_rles = multi_rle_encode(cur_seg)
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                out_pred_rows[m] += [{'ImageId': c_img_name, 'EncodedPixels': c_rle}]
        else:
            out_pred_rows[m] += [{'ImageId': c_img_name, 'EncodedPixels': None}]
    gc.collect()
#     os.system('echo '+str(i))
#     i += 1

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
submission_df1 = pd.DataFrame(out_pred_rows[0])[['ImageId', 'EncodedPixels']]
submission_df1.to_csv('fpn35.csv', index=False)
submission_df1.sample(3)

,ImageId,EncodedPixels
1087,9349f17a7.jpg,1 589824
13790,deb09b00a.jpg,1 589824
15801,4d52c8656.jpg,1 28360 28369 760 29137 759 29905 758 30673 75...


In [6]:
submission_df2 = pd.DataFrame(out_pred_rows[1])[['ImageId', 'EncodedPixels']]
submission_df2.to_csv('unet35.csv', index=False)
submission_df2.sample(3)

,ImageId,EncodedPixels
6421,443e09d4a.jpg,None
21808,0ddc42ee1.jpg,436759 3 437526 7 438293 10 439062 10 439831 1...
8806,670c38610.jpg,229307 3 230072 7 230839 9 231606 11 232373 13...
